In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from datetime import datetime, timedelta
import pmdarima as pm
import concurrent.futures
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import threading

/Users/zhuhongyun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/zhuhongyun/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
warnings.filterwarnings("ignore", message="A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.")
warnings.filterwarnings("ignore", message="No supported index is available. Prediction results will be given with an integer index beginning at `start`.")
warnings.filterwarnings("ignore", message="No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.")
warnings.filterwarnings("ignore", message="Non-invertible starting MA parameters found. Using zeros as starting parameters.")
warnings.filterwarnings("ignore", message="Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.")

In [3]:
data = pd.read_csv("tape_result.csv")

In [4]:
data_copy = data[['Time','Weighted_Price']].copy()

In [5]:
data_copy['Time'] = pd.to_datetime(data_copy['Time'], unit='s')

In [6]:
data_copy.set_index('Time',inplace=True)

In [7]:
data_copy

,Weighted_Price
Time,
1970-01-01 00:00:04.836,142.0
1970-01-01 00:00:06.913,129.0
1970-01-01 00:00:09.052,130.0
1970-01-01 00:00:09.300,128.0
1970-01-01 00:00:12.989,132.0
...,...
1970-01-01 08:29:52.753,133.0
1970-01-01 08:29:53.528,133.0
1970-01-01 08:29:54.458,134.0


In [8]:
# Arima 5s multithreading version
start_time = datetime.strptime("1970-01-01 00:00:00.000", "%Y-%m-%d %H:%M:%S.%f")
pred = pd.DataFrame([])

def custom_find_nearest(time_index, target_time):

    # Compute time difference
    time_deltas = abs(time_index - target_time)
    # Find the index with the smallest time difference
    closest_index = time_deltas.argmin()
    return closest_index

def process_data(i):
    current_start = start_time + timedelta(seconds=i*10)
    current_end = current_start + timedelta(seconds=600)
    print(f"{current_start} - {current_end}"+" start")
    print("threads counts:", threading.active_count())
    data_copy_slice = data_copy.loc[current_start:current_end]
    data_copy_slice2 = data_copy.loc[current_end-timedelta(seconds=10):current_end+timedelta(seconds=10)]

#     print(data_copy_slice2)

    model = ARIMA(data_copy_slice, order=(1, 2, 1))
    fitted_model = model.fit()
    forecast = fitted_model.forecast(steps=10)
    forecast_values = forecast.values 
    forecast_index = [current_end + timedelta(seconds=x*1) for x in range(1, 11)]  # Create a new time index
    forecast_df = pd.DataFrame(forecast_values, index=forecast_index, columns=['Forecasted_Price'])
    print(f"{current_start} - {current_end}"+" end")
    
    max_price = forecast_df['Forecasted_Price'].max()
    min_price = forecast_df['Forecasted_Price'].min()
#     print(max_price)
#     print(min_price)
    max_time = forecast_df['Forecasted_Price'].idxmax()
    min_time = forecast_df['Forecasted_Price'].idxmin()
    max_time = pd.to_datetime(max_time)
    min_time = pd.to_datetime(min_time)
    closest_max_time_index = custom_find_nearest(data_copy_slice2.index, max_time)
    closest_min_time_index = custom_find_nearest(data_copy_slice2.index, min_time)
    closest_max_time_price = data_copy_slice2.iloc[closest_max_time_index]['Weighted_Price']
    closest_min_time_price = data_copy_slice2.iloc[closest_min_time_index]['Weighted_Price']
#     print(closest_max_time_price)
#     print(closest_min_time_price)
    high_deviation = (abs(max_price - closest_max_time_price) / closest_max_time_price)
    low_deviation = (abs(min_price - closest_min_time_price ) / closest_min_time_price )

    # Compare the time when the highest point and the lowest point occur and return the decision value
    decision = 1 if min_time < max_time else 2
    
    act_buy = False if min_price <  (1-low_deviation)*closest_min_time_price  else True
    
    act_sell = False if max_price >  (1+high_deviation)*closest_max_time_price else True
    
    
    
    result = pd.DataFrame({
        'Highest_Price': max_price,
        'Lowest_Price': min_price,
        'High_deviation': high_deviation,
        'Low_deviation': low_deviation,
        'Decision': decision,
        'act_buy':act_buy,
        'act_sell':act_sell
    },  index=[current_end]) 

#     print(result)
    return result

# with concurrent.futures.ThreadPoolExecutor() as executor:
#     futures = [executor.submit(process_data, i) for i in range(3000)]
with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
    futures = [executor.submit(process_data, i) for i in range(3000)]
    

    for future in concurrent.futures.as_completed(futures):
        pred = pd.concat([pred, future.result()])
        
sorted_pred = pred.sort_index()        

1970-01-01 00:00:00 - 1970-01-01 00:10:00 start
threads counts: 10
1970-01-01 00:00:10 - 1970-01-01 00:10:10 start
threads counts: 10
1970-01-01 00:00:20 - 1970-01-01 00:10:20 start
threads counts: 11
1970-01-01 00:00:30 - 1970-01-01 00:10:30 start
threads counts: 12
1970-01-01 00:00:40 - 1970-01-01 00:10:40 start
threads counts: 13
1970-01-01 00:00:50 - 1970-01-01 00:10:50 start
threads counts: 14
1970-01-01 00:01:00 - 1970-01-01 00:11:00 start
threads counts: 15
1970-01-01 00:01:10 - 1970-01-01 00:11:10 start
threads counts: 16
1970-01-01 00:01:20 - 1970-01-01 00:11:20 start
threads counts: 17
1970-01-01 00:01:30 - 1970-01-01 00:11:30 start
threads counts: 18
1970-01-01 00:01:40 - 1970-01-01 00:11:40 start
threads counts: 19
1970-01-01 00:01:50 - 1970-01-01 00:11:50 start
threads counts: 20
1970-01-01 00:02:00 - 1970-01-01 00:12:00 start1970-01-01 00:02:10 - 1970-01-01 00:12:10 start
threads counts: 22

threads counts: 22
1970-01-01 00:02:20 - 1970-01-01 00:12:20 start
threads counts

1970-01-01 00:12:30 - 1970-01-01 00:22:30 end1970-01-01 00:06:10 - 1970-01-01 00:16:10 end
1970-01-01 00:19:30 - 1970-01-01 00:29:30 start
threads counts: 108

1970-01-01 00:19:40 - 1970-01-01 00:29:40 start
threads counts: 108
1970-01-01 00:11:40 - 1970-01-01 00:21:40 end
1970-01-01 00:19:50 - 1970-01-01 00:29:50 start
threads counts: 108
1970-01-01 00:13:20 - 1970-01-01 00:23:20 end
1970-01-01 00:20:00 - 1970-01-01 00:30:00 start
threads counts: 108
1970-01-01 00:06:50 - 1970-01-01 00:16:50 end
1970-01-01 00:20:10 - 1970-01-01 00:30:10 start
threads counts: 108
1970-01-01 00:03:20 - 1970-01-01 00:13:20 end
1970-01-01 00:20:20 - 1970-01-01 00:30:20 start
threads counts: 108
1970-01-01 00:04:50 - 1970-01-01 00:14:50 end
1970-01-01 00:20:30 - 1970-01-01 00:30:30 start
threads counts: 108
1970-01-01 00:09:50 - 1970-01-01 00:19:50 end
1970-01-01 00:20:40 - 1970-01-01 00:30:40 start
threads counts: 108
1970-01-01 00:09:20 - 1970-01-01 00:19:20 end
1970-01-01 00:11:30 - 1970-01-01 00:21:30 

1970-01-01 00:04:10 - 1970-01-01 00:14:10 end
1970-01-01 00:31:30 - 1970-01-01 00:41:30 start
threads counts: 108
1970-01-01 00:06:20 - 1970-01-01 00:16:20 end
1970-01-01 00:31:40 - 1970-01-01 00:41:40 start
threads counts: 108
1970-01-01 00:13:00 - 1970-01-01 00:23:00 end
1970-01-01 00:31:50 - 1970-01-01 00:41:50 start
threads counts: 108
1970-01-01 00:05:10 - 1970-01-01 00:15:10 end
1970-01-01 00:32:00 - 1970-01-01 00:42:00 start
threads counts: 108
1970-01-01 00:01:10 - 1970-01-01 00:11:10 end
1970-01-01 00:32:10 - 1970-01-01 00:42:10 start
threads counts: 108
1970-01-01 00:00:10 - 1970-01-01 00:10:10 end
1970-01-01 00:32:20 - 1970-01-01 00:42:20 start
threads counts: 108
1970-01-01 00:08:40 - 1970-01-01 00:18:40 end
1970-01-01 00:32:30 - 1970-01-01 00:42:30 start
threads counts: 108
1970-01-01 00:16:30 - 1970-01-01 00:26:30 end
1970-01-01 00:32:40 - 1970-01-01 00:42:40 start
threads counts: 108
1970-01-01 00:14:50 - 1970-01-01 00:24:50 end
1970-01-01 00:32:50 - 1970-01-01 00:42:50 

1970-01-01 00:30:50 - 1970-01-01 00:40:50 end
1970-01-01 00:44:00 - 1970-01-01 00:54:00 start
threads counts: 108
1970-01-01 00:24:50 - 1970-01-01 00:34:50 end
1970-01-01 00:44:10 - 1970-01-01 00:54:10 start
threads counts: 108
1970-01-01 00:25:10 - 1970-01-01 00:35:10 end
1970-01-01 00:44:20 - 1970-01-01 00:54:20 start
threads counts: 108
1970-01-01 00:23:00 - 1970-01-01 00:33:00 end
1970-01-01 00:44:30 - 1970-01-01 00:54:30 start
threads counts: 108
1970-01-01 00:24:30 - 1970-01-01 00:34:30 end
1970-01-01 00:44:40 - 1970-01-01 00:54:40 start
threads counts: 108
1970-01-01 00:19:50 - 1970-01-01 00:29:50 end
1970-01-01 00:44:50 - 1970-01-01 00:54:50 start
threads counts: 108
1970-01-01 00:17:40 - 1970-01-01 00:27:40 end
1970-01-01 00:45:00 - 1970-01-01 00:55:00 start
threads counts: 108
1970-01-01 00:31:10 - 1970-01-01 00:41:10 end
1970-01-01 00:45:10 - 1970-01-01 00:55:10 start
threads counts: 108
1970-01-01 00:29:20 - 1970-01-01 00:39:20 end
1970-01-01 00:29:30 - 1970-01-01 00:39:30 

1970-01-01 00:40:40 - 1970-01-01 00:50:40 end
1970-01-01 00:56:30 - 1970-01-01 01:06:30 start
threads counts: 108
1970-01-01 00:40:10 - 1970-01-01 00:50:10 end
1970-01-01 00:56:40 - 1970-01-01 01:06:40 start
threads counts: 108
1970-01-01 00:34:40 - 1970-01-01 00:44:40 end
1970-01-01 00:56:50 - 1970-01-01 01:06:50 start
threads counts: 108
1970-01-01 00:45:30 - 1970-01-01 00:55:30 end
1970-01-01 00:57:00 - 1970-01-01 01:07:00 start
threads counts: 108
1970-01-01 00:41:10 - 1970-01-01 00:51:10 end
1970-01-01 00:57:10 - 1970-01-01 01:07:10 start
threads counts: 108
1970-01-01 00:42:10 - 1970-01-01 00:52:10 end
1970-01-01 00:43:10 - 1970-01-01 00:53:10 end
1970-01-01 00:57:20 - 1970-01-01 01:07:20 start
threads counts: 108
1970-01-01 00:57:30 - 1970-01-01 01:07:30 start
threads counts: 108
1970-01-01 00:34:20 - 1970-01-01 00:44:20 end
1970-01-01 00:57:40 - 1970-01-01 01:07:40 start
threads counts: 108
1970-01-01 00:44:40 - 1970-01-01 00:54:40 end
1970-01-01 00:57:50 - 1970-01-01 01:07:50 

...

1970-01-01 07:16:30 - 1970-01-01 07:26:30 end
1970-01-01 07:30:00 - 1970-01-01 07:40:00 start
threads counts: 108
1970-01-01 07:02:40 - 1970-01-01 07:12:40 end
1970-01-01 07:30:10 - 1970-01-01 07:40:10 start
threads counts: 108
1970-01-01 07:19:00 - 1970-01-01 07:29:00 end
1970-01-01 07:30:20 - 1970-01-01 07:40:20 start
threads counts: 108
1970-01-01 07:14:50 - 1970-01-01 07:24:50 end
1970-01-01 07:30:30 - 1970-01-01 07:40:30 start
threads counts: 108
1970-01-01 07:10:40 - 1970-01-01 07:20:40 end
1970-01-01 07:30:40 - 1970-01-01 07:40:40 start
threads counts: 108
1970-01-01 07:18:20 - 1970-01-01 07:28:20 end
1970-01-01 07:30:50 - 1970-01-01 07:40:50 start
threads counts: 108
1970-01-01 07:09:10 - 1970-01-01 07:19:10 end
1970-01-01 07:10:20 - 1970-01-01 07:20:20 end
1970-01-01 07:31:00 - 1970-01-01 07:41:00 start
threads counts: 108
1970-01-01 07:31:10 - 1970-01-01 07:41:10 start
threads counts: 108
1970-01-01 07:16:10 - 1970-01-01 07:26:10 end
1970-01-01 07:11:50 - 1970-01-01 07:21:50 

1970-01-01 07:29:00 - 1970-01-01 07:39:00 end
1970-01-01 07:42:30 - 1970-01-01 07:52:30 start
threads counts: 108
1970-01-01 07:23:50 - 1970-01-01 07:33:50 end
1970-01-01 07:42:40 - 1970-01-01 07:52:40 start
threads counts: 108
1970-01-01 07:24:10 - 1970-01-01 07:34:10 end
1970-01-01 07:42:50 - 1970-01-01 07:52:50 start
threads counts: 108
1970-01-01 07:24:00 - 1970-01-01 07:34:00 end
1970-01-01 07:43:00 - 1970-01-01 07:53:00 start
threads counts: 108
1970-01-01 07:27:20 - 1970-01-01 07:37:20 end
1970-01-01 07:43:10 - 1970-01-01 07:53:10 start
threads counts: 108
1970-01-01 07:29:40 - 1970-01-01 07:39:40 end
1970-01-01 07:43:20 - 1970-01-01 07:53:20 start
threads counts: 108
1970-01-01 07:26:20 - 1970-01-01 07:36:20 end
1970-01-01 07:43:30 - 1970-01-01 07:53:30 start
threads counts: 108
1970-01-01 07:30:00 - 1970-01-01 07:40:00 end
1970-01-01 07:43:40 - 1970-01-01 07:53:40 start
threads counts: 108
1970-01-01 07:31:20 - 1970-01-01 07:41:20 end
1970-01-01 07:43:50 - 1970-01-01 07:53:50 

1970-01-01 07:42:00 - 1970-01-01 07:52:00 end1970-01-01 07:32:20 - 1970-01-01 07:42:20 end
1970-01-01 07:54:40 - 1970-01-01 08:04:40 start
threads counts: 108

1970-01-01 07:54:50 - 1970-01-01 08:04:50 start
threads counts: 108
1970-01-01 07:39:50 - 1970-01-01 07:49:50 end
1970-01-01 07:55:00 - 1970-01-01 08:05:00 start
threads counts: 108
1970-01-01 07:41:30 - 1970-01-01 07:51:30 end
1970-01-01 07:55:10 - 1970-01-01 08:05:10 start
threads counts: 108
1970-01-01 07:42:30 - 1970-01-01 07:52:30 end
1970-01-01 07:55:20 - 1970-01-01 08:05:20 start1970-01-01 07:36:40 - 1970-01-01 07:46:40 end
1970-01-01 07:55:30 - 1970-01-01 08:05:30 start
threads counts: 108

threads counts: 108
1970-01-01 07:39:30 - 1970-01-01 07:49:30 end
1970-01-01 07:40:30 - 1970-01-01 07:50:30 end
1970-01-01 07:55:40 - 1970-01-01 08:05:40 start
threads counts: 108
1970-01-01 07:55:50 - 1970-01-01 08:05:50 start
threads counts: 108
1970-01-01 07:40:20 - 1970-01-01 07:50:20 end
1970-01-01 07:56:00 - 1970-01-01 08:06:00 

1970-01-01 08:07:00 - 1970-01-01 08:17:00 start
threads counts: 108
1970-01-01 07:51:40 - 1970-01-01 08:01:40 end
1970-01-01 08:07:10 - 1970-01-01 08:17:10 start
threads counts: 108
1970-01-01 07:54:00 - 1970-01-01 08:04:00 end
1970-01-01 08:07:20 - 1970-01-01 08:17:20 start
threads counts: 108
1970-01-01 07:49:30 - 1970-01-01 07:59:30 end
1970-01-01 08:07:30 - 1970-01-01 08:17:30 start
threads counts: 108
1970-01-01 07:55:20 - 1970-01-01 08:05:20 end
1970-01-01 08:07:40 - 1970-01-01 08:17:40 start
threads counts: 108
1970-01-01 07:54:20 - 1970-01-01 08:04:20 end
1970-01-01 08:07:50 - 1970-01-01 08:17:50 start
threads counts: 108
1970-01-01 07:46:50 - 1970-01-01 07:56:50 end
1970-01-01 08:08:00 - 1970-01-01 08:18:00 start
threads counts: 108
1970-01-01 07:48:00 - 1970-01-01 07:58:00 end
1970-01-01 08:08:10 - 1970-01-01 08:18:10 start
threads counts: 108
1970-01-01 07:47:30 - 1970-01-01 07:57:30 end
1970-01-01 07:47:50 - 1970-01-01 07:57:50 end
1970-01-01 08:08:20 - 1970-01-01 08:18:20 

1970-01-01 08:03:20 - 1970-01-01 08:13:20 end
1970-01-01 08:19:10 - 1970-01-01 08:29:10 start
threads counts: 108
1970-01-01 08:01:20 - 1970-01-01 08:11:20 end
1970-01-01 08:19:20 - 1970-01-01 08:29:20 start
threads counts: 108
1970-01-01 07:58:30 - 1970-01-01 08:08:30 end
1970-01-01 08:19:30 - 1970-01-01 08:29:30 start
threads counts: 108
1970-01-01 08:02:40 - 1970-01-01 08:12:40 end
1970-01-01 08:19:40 - 1970-01-01 08:29:40 start
threads counts: 108
1970-01-01 08:07:10 - 1970-01-01 08:17:10 end
1970-01-01 08:19:50 - 1970-01-01 08:29:50 start
threads counts: 108
1970-01-01 08:07:20 - 1970-01-01 08:17:20 end
1970-01-01 08:02:20 - 1970-01-01 08:12:20 end
1970-01-01 08:01:30 - 1970-01-01 08:11:30 end
1970-01-01 08:06:00 - 1970-01-01 08:16:00 end1970-01-01 08:07:40 - 1970-01-01 08:17:40 end

1970-01-01 08:00:00 - 1970-01-01 08:10:00 end
1970-01-01 08:04:20 - 1970-01-01 08:14:20 end
1970-01-01 08:09:00 - 1970-01-01 08:19:00 end
1970-01-01 08:07:50 - 1970-01-01 08:17:50 end
1970-01-01 08:09

In [113]:
sorted_pred

,Highest_Price,Lowest_Price,High_deviation,Low_deviation,Decision,act_buy,act_sell
1970-01-01 00:10:00,140.006595,140.001575,0.000047,0.007081,1,True,True
1970-01-01 00:10:05,140.602022,140.512431,0.016769,0.003660,1,True,True
1970-01-01 00:10:10,142.587120,142.356782,0.040782,0.002513,1,True,True
1970-01-01 00:10:15,136.894776,136.803257,0.014035,0.013357,1,True,True
1970-01-01 00:10:20,135.796414,135.647107,0.030026,0.031092,2,True,True
1970-01-01 00:10:25,139.102656,138.845728,0.006410,0.008245,1,True,True
1970-01-01 00:10:30,139.105895,138.847311,0.013433,0.015267,1,True,True
1970-01-01 00:10:35,140.224412,140.174726,0.001603,0.001248,2,True,True
1970-01-01 00:10:40,142.496770,142.341827,0.040122,0.024042,1,True,True
1970-01-01 00:10:45,137.219368,137.170150,0.001601,0.034013,2,True,True


In [118]:
sorted_pred.to_csv('sorted_pred.csv')

In [33]:
data_copy_slice = data_copy.loc['1970-01-01 00:00:00.000':'1970-01-01 08:30:00']

In [11]:
initial_capital = 5000
transaction_num = 1
hold_stocks = False
transaction_result = []
stock_num = 0


for index, row in sorted_pred.iterrows():
    operation = row['Decision']
    highest_price = row['Highest_Price']
    lowest_price = row['Lowest_Price']
    act_buy = row['act_buy']
    act_sell = row['act_sell']
    
    if highest_price-lowest_price >= 0.01 and operation == 1:
        
        if  act_buy == True and act_sell == True  and initial_capital >= highest_price:

            initial_capital -= transaction_num * lowest_price
            stock_num += transaction_num
            initial_capital += stock_num*highest_price
            sell_num = stock_num
            stock_num = 0
            if stock_num != 0:
                hold_stocks = True
            else:
                hold_stocks = False
            transaction_result.append({'operation': 'buy first and sell all', 'buy price':lowest_price,'sell price':highest_price ,'sell number':sell_num ,'buy_num':transaction_num})

        if  act_buy == False and act_sell == True   and hold_stocks == True :

            initial_capital += stock_num*highest_price
            sell_num = stock_num
            stock_num = 0
            if stock_num != 0:
                hold_stocks = True
            else:
                hold_stocks = False
            transaction_result.append({'operation': 'buy faild only sell', 'buy price': 0,'sell price': highest_price,'sell number':sell_num ,'buy_num': 0 })

        if  act_buy == False and act_sell == True  and initial_capital >= highest_price :
            initial_capital -= transaction_num * lowest_price
            stock_num += transaction_num
            if stock_num != 0:
                hold_stocks = True
            else:
                hold_stocks = False
            transaction_result.append({'operation': 'sell faild only buy', 'buy price':lowest_price ,'sell price': 0,'sell number':0 ,'buy_num': transaction_num })

        if  act_buy == False and act_sell == False:
            transaction_result.append({'operation': 'sell faild and buy faild', 'buy price':0 ,'sell price': 0,'sell number':0 ,'buy_num': 0 })

transaction_result_df = pd.DataFrame(transaction_result)
print(transaction_result_df)

# Calculate final capital and profit
final_capital = initial_capital
profit = final_capital - 5000
Profitability = profit/5000
profitability_percent = "{:.4f}%".format(Profitability * 100)
print(f'capital at closing：{final_capital}')
print(f'total_profit：{profit}')
print(f'Profitability：{profitability_percent}')


                   operation   buy price  sell price  sell number  buy_num
0     buy first and sell all  140.001575  140.012944            1        1
1     buy first and sell all  142.356782  142.721707            1        1
2     buy first and sell all  138.847311  139.105895            1        1
3     buy first and sell all  140.969056  141.173878            1        1
4     buy first and sell all  142.832932  143.027550            1        1
...                      ...         ...         ...          ...      ...
1246  buy first and sell all  137.130861  137.225473            1        1
1247  buy first and sell all  137.337333  137.537437            1        1
1248  buy first and sell all  136.103279  136.178240            1        1
1249  buy first and sell all  137.320730  137.459410            1        1
1250  buy first and sell all  137.923368  137.966701            1        1

[1251 rows x 5 columns]
capital at closing：5176.589883106936
total_profit：176.5898831069362
Profita